In [ ]:
from mpcontribs.client import Client
from pandas import DataFrame

client = Client()

In [ ]:
client.delete_contributions("sandbox")

In [ ]:
client.get_project("sandbox").get("columns")

In [ ]:
data = [['tom', 10], ['nick', 15], ['juli', 14]]
df = DataFrame(data, columns=['Name', 'Age'])
df.set_index("Name", inplace=True)
df

In [ ]:
contributions = [{
    "project": "sandbox",
    "identifier": "mp-4",
    "data": {
        "a": "3 eV",
        "b": {"c": "hello", "d": 5},
        "d.e.f": "nest via dot-notation"
    },
    "tables": [df]
}, {
    "project": "sandbox",
    "identifier": "mp-6",
    "data": {
        "a": "4 eV",
        "b": {"c": "what", "d": 6},
        "d.e.f": "duh"
    },
    "tables": [df]
}]
client.submit_contributions(contributions, ignore_dupes=True)

In [ ]:
client.get_project("sandbox").get("columns")